In [1]:
import os
import pandas as pd
def gather_teamaverage(start_year,end_year):
    all_years=[]
    for year in range(2014,end_year+1):
        file = "game_report/year_files/"+str(year)+"_games.csv"
        df = pd.read_csv(file)
        all_years.append(df)
        
    return pd.concat(all_years)

df= gather_teamaverage(2014,2025)


In [2]:
df

,PLAYER_ID,PLAYER_NAME,NICKNAME,TEAM_ID,TEAM_ABBREVIATION,AGE,GP,W,L,W_PCT,...,FGA_10_14,CATCH_SHOOT_FGA,CATCH_SHOOT_FG3_PCT,CATCH_SHOOT_FG3A,CATCH_SHOOT_FG3M,CATCH_SHOOT_EFG_PCT,CATCH_SHOOT_FGM,year,date,playoffs
0,201166,Aaron Brooks,Aaron,1610612743,DEN,29.0,1,1,0,1.0,...,0.0,0,NaN,NaN,NaN,NaN,0,2014,20131030,False
1,1733,Al Harrington,Al,1610612764,WAS,34.0,1,0,1,0.0,...,0.0,1,0.000,1.0,0.0,0.00,0,2014,20131030,False
2,201143,Al Horford,Al,1610612737,ATL,28.0,1,0,1,0.0,...,2.0,3,NaN,NaN,NaN,1.00,3,2014,20131030,False
3,2744,Al Jefferson,Al,1610612766,CHA,29.0,1,0,1,0.0,...,5.0,4,NaN,NaN,NaN,0.50,2,2014,20131030,False
4,202329,Al-Farouq Aminu,Al-Farouq,1610612740,NOP,23.0,1,0,1,0.0,...,0.0,1,NaN,0.0,0.0,0.00,0,2014,20131030,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7363,1631117,Walker Kessler,Walker,1610612762,UTA,23.0,1,1,0,1.0,...,0.0,0,NaN,NaN,NaN,NaN,0,2025,20241206,False
7364,1628976,Wendell Carter Jr.,Wendell,1610612753,ORL,25.0,1,0,1,0.0,...,0.0,2,0.500,2.0,1.0,0.75,1,2025,20241206,False
7365,1642258,Zaccharie Risacher,Zaccharie,1610612737,ATL,19.0,1,1,0,1.0,...,1.0,3,0.333,3.0,1.0,0.50,1,2025,20241206,False
7366,1628380,Zach Collins,Zach,1610612759,SAS,27.0,1,0,1,0.0,...,0.0,1,0.000,1.0,0.0,0.00,0,2025,20241206,False


In [5]:
year=df[df.year==2021]
year=year.drop_duplicates(subset=['PLAYER_ID','TEAM_ID','date'])
year.groupby(['TEAM_ID','TEAM_ABBREVIATION']).sum()['POINTS']

/tmp/ipykernel_11993/4125419149.py:3: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  year.groupby(['TEAM_ID','TEAM_ABBREVIATION']).sum()['POINTS']


TEAM_ID     TEAM_ABBREVIATION
1610612737  ATL                  8590.0
1610612738  BOS                  7847.0
1610612739  CLE                  7137.0
1610612740  NOP                  8136.0
1610612741  CHI                  8953.0
1610612742  DAL                  8194.0
1610612743  DEN                  8732.0
1610612744  GSW                  8016.0
1610612745  HOU                  7164.0
1610612746  LAC                  8038.0
1610612747  LAL                  8486.0
1610612748  MIA                  8010.0
1610612749  MIL                  8960.0
1610612750  MIN                  8073.0
1610612751  BKN                  8804.0
1610612752  NYK                  7763.0
1610612753  ORL                  6117.0
1610612754  IND                  8344.0
1610612755  PHI                  8509.0
1610612756  PHX                  8323.0
1610612757  POR                  8390.0
1610612758  SAC                  8357.0
1610612759  SAS                  8008.0
1610612760  OKC                  7197.0
1610612761